In [1]:
addprocs(3)
nprocs()

4

In [2]:
@everywhere include("../src/UNSflow.jl")
#@everywhere include("../src/UNSflow.jl")
#@everywhere using UNSflow

LoadError: LoadError: On worker 2:
LoadError: InitError: PyError (:PyImport_ImportModule) None

 in pyerr_check at /Users/kiran/.julia/v0.5/PyCall/src/exception.jl:56 [inlined]
 in pyerr_check at /Users/kiran/.julia/v0.5/PyCall/src/exception.jl:61 [inlined]
 in macro expansion at /Users/kiran/.julia/v0.5/PyCall/src/exception.jl:81 [inlined]
 in pyimport at /Users/kiran/.julia/v0.5/PyCall/src/PyCall.jl:387
 in __init__ at /Users/kiran/.julia/v0.5/PyPlot/src/PyPlot.jl:256
 in _require_from_serialized at ./loading.jl:190
 in _require_search_from_serialized at ./loading.jl:217
 in jlcall__require_search_from_serialized_34134 at /opt/julia/usr/lib/julia/sys.dylib:?
 in require at ./loading.jl:371
 in jlcall_require_22382 at /opt/julia/usr/lib/julia/sys.dylib:?
 in include_string at ./loading.jl:441
 in jlcall_include_string_35097 at /opt/julia/usr/lib/julia/sys.dylib:?
 in include_from_node1 at ./loading.jl:491
 in jlcall_include_from_node1_20159 at /opt/julia/usr/lib/julia/sys.dylib:?
 in 

In [3]:
@everywhere include("../src/UNSflow.jl")
@everywhere using UNSflow


Use "(eld::EldUpDef)(...)" instead.

Use "(eld::EldUptstartDef)(...)" instead.

Use "(cons::ConstDef)(...)" instead.

Use "(eld::EldRampReturnDef)(...)" instead.

Use "(eld::EldUpDef)(...)" instead.

Use "(kin::SinDef)(...)" instead.

Use "(kin::CosDef)(...)" instead.

Use "(eld::EldUptstartDef)(...)" instead.

Use "(cons::ConstDef)(...)" instead.

Use "(eld::EldRampReturnDef)(...)" instead.

Use "(eld::EldUpDef)(...)" instead.

Use "(kin::SinDef)(...)" instead.

Use "(kin::CosDef)(...)" instead.

Use "(eld::EldUptstartDef)(...)" instead.

Use "(cons::ConstDef)(...)" instead.

Use "(eld::EldRampReturnDef)(...)" instead.

Use "(kin::SinDef)(...)" instead.

Use "(kin::CosDef)(...)" instead.
  likely near /Users/kiran/Documents/Projects/UNSflow/src/types.jl:294
  likely near /Users/kiran/Documents/Projects/UNSflow/src/types.jl:294
  likely near /Users/kiran/Documents/Projects/UNSflow/src/types.jl:486
 instead.
  likely near /Users/kiran/Documents/Projects/UNSflow/src/types.jl:294
  likel

# Recreate Antonio 3D results using a strip theory approach and LDVM

### basic LLT correction - circulation goes to zero at the wingtips. The LDVM runs are done in parallel  


In [10]:
#Construct dimensionless quantities from given values
c_d = 0.2
b_d = 6
AR = b_d/c_d

u_d = 10
f_d = 0.8889
k = 2*pi*f_d*c_d/(2*u_d)

#_dAll values below are nondimensional
c = 1
u = 1
w = 2*k
T = (2*pi/w)
ncyc = 8
t_tot = ncyc*T 

modedata = readdlm("../test/anto_mode1.dat");
mode_spl = Spline1D(modedata[:,3],modedata[:,4])

n_span = 12
n_bterm = 40

psi = zeros(n_span)
dpsi = pi/n_span

for i = 1:n_span
    psi[i] = (real(i)-0.5)*dpsi
end

scale = 0.5

#Run LDVM at all these locations:
alpha_amp = 5*pi/180

h_amp = zeros(n_span)
for i = 1:n_span   
    y_d = -b_d*cos(psi[i])/2.
    h_amp[i] = evaluate(mode_spl,y_d)*scale/c_d
end

dtstar = min(0.015*8,0.015*0.2*4/(k*maximum(h_amp)))
nsteps =round(Int,t_tot/dtstar)+1

alphadef = ConstDef(alpha_amp)
udef = ConstDef(u)
pvt = 0.0 #Doesnt matter, no pitch
lespcrit = [21;] #high value to turn off LEV shedding



1-element Array{Int64,1}:
 21

In [11]:
nsteps

3751

In [12]:
#Time parallel and serial runs

#Parallel

W_mat = Array{Float64,2}[]
W_surf = TwoDSurf[]
W_curfield = TwoDFlowField[]

#Create an aray of inputs for use in pmap. 
#ldvm_in = Tuple{TwoDSurf,TwoDFlowField,Int64,Float64}[]
surf_in = TwoDSurf[]
field_in = TwoDFlowField[]
n_in = Int64[]
dt_in = Float64[]
del_in = DelVortDef[]

for i = 1:Int(n_span/2)
    hdef = CosDef(0., h_amp[i], w, 0.)
    full_kinem = KinemDef(alphadef, hdef, udef)
    surf = TwoDSurf("FlatPlate", pvt, full_kinem, lespcrit)
    curfield = TwoDFlowField()
    del = DelVortDef(1, 500, 10)
    push!(surf_in,surf)
    push!(field_in, curfield)
    push!(n_in,nsteps,)
    push!(dt_in,dtstar)
    push!(del_in,del)
end

@time fullsol = pmap((a1,a2,a3,a4,a5)->ldvm(a1,a2,a3,a4),surf_in,field_in,n_in,dt_in,del_in)

for i = 1:Int(n_span/2)
    push!(W_mat,fullsol[i][1])
    push!(W_surf, fullsol[i][2])
    push!(W_curfield, fullsol[i][3])
end
    



856.849131 seconds (750.54 k allocations: 27.560 MB, 0.00% gc time)


In [ ]:
W_mat


In [ ]:
#Mirror image for the rest of the span
for i = Int(n_span/2)+1:n_span
    mt = W_mat[n_span - i + 1]
    st = W_surf[n_span - i + 1]
    ct = W_curfield[n_span - i + 1]
    push!(W_mat, mt)
    push!(W_surf, st)
    push!(W_curfield, ct)
end

In [ ]:
lhs = zeros(n_span,n_bterm)
rhs = zeros(n_span)
b_coeff = zeros(nsteps,n_bterm)
sp_gam = zeros(nsteps,n_span)

dt = W_mat[1][2,1] - W_mat[1][1,1]

cnc_f = zeros(nsteps)
cnnc_f = zeros(nsteps)



In [ ]:
bdot = zeros(nsteps,n_bterm)

for i = 1:nsteps
    for j = 1:n_span
        for n = 1:n_bterm
            lhs[j,n] = sin(n*psi[j])*(sin(psi[j]) + (n*pi/(2*AR)))
        end
        rhs[j] = pi*sin(psi[j])*W_mat[j][i,9]/(2*AR)
    end
        
    b_coeff[i,:] = \(lhs, rhs)
    if i >= 2   
        bdot[i,:] = (b_coeff[i,:] - b_coeff[i-1,:])/dt
    end
       
end
        

In [ ]:
a03d = zeros(nsteps,n_span)
cd_ind = zeros(nsteps)
a0dot3d = zeros(nsteps,n_span)
gam_other = zeros(nsteps,n_span)

for i = 1:nsteps
    for j = 1:n_span
         sp_gam[i,j] = 0
         for n = 1:n_bterm
             sp_gam[i,j] = sp_gam[i,j] + 2*b_d*u*b_coeff[i,n]*sin(n*psi[j])
         end
    end  
end

for i = 1:nsteps
    cd_ind[i] = 0
    for n = 1:n_bterm
        cd_ind[i] = cd_ind[i] + real(n)*b_coeff[i,n]^2
    end
    cd_ind[i] = cd_ind[i]*pi*AR
    for j = 1:n_span
        a03d[i,j] = 0
        for n = 1:n_bterm
            a03d[i,j] = a03d[i,j] - real(n)*b_coeff[i,n]*sin(n*psi[j])/sin(psi[j])
            a0dot3d[i,j] = a0dot3d[i,j] - real(n)*bdot[i,n]*sin(n*psi[j])/sin(psi[j])
        end
        gam_other[i,j] = u*c_d*pi*(W_mat[j][i,9] + a03d[i,j])
    end
end



In [ ]:
plot(sp_gam[10,1:n_span])
plot(gam_other[10,:])
plot(a03d[10,:])
plot(a0dot3d[10,:])

In [ ]:
W_alpha = zeros(nsteps,n_span)
W_h = zeros(nsteps,n_span)
W_hdot = zeros(nsteps,n_span)
W_u = zeros(nsteps,n_span)
W_u[:,:] = 1
W_alpha[:,:] = 5*pi/180
for i = 1:nsteps
    for j = 1:n_span
        hdef = CosDef(0., h_amp[j], w, 0.)
        tt = W_mat[1][i,1]    
        W_h[i,j] = hdef(tt)*c
        W_hdot[i,j] = ForwardDiff.derivative(hdef,tt)*u
    end
end


In [ ]:
W_cn = zeros(nsteps)
W_cs = zeros(nsteps)
W_cl = zeros(nsteps)
W_cd = zeros(nsteps)
W_cdi = zeros(nsteps)

cn3d = zeros(nsteps, n_span)
cs3d = zeros(nsteps, n_span)
cl3d = zeros(nsteps, n_span)
cd3d = zeros(nsteps, n_span)

for i = 1:nsteps
    W_cn[i] = 0
    W_cs[i] = 0
    for j = 1:n_span
        cn3d[i,j] = W_mat[j][i,10] + (2*pi/u)*(W_u[i,j]*cos(W_alpha[i,j]) + W_hdot[i,j]*sin(W_alpha[i,j]))*a03d[1,j] + (2*pi*c/u)*(3*a0dot3d[i,j]/4)
        cs3d[i,j] = W_mat[j][i,11] + 2*pi*a03d[i,j]^2
        cl3d[i,j] = cn3d[i,j]*cos(W_alpha[i,j]) + cs3d[i,j]*sin(W_alpha[i,j])
        cd3d[i,j] = cn3d[i,j]*sin(W_alpha[i,j]) - cs3d[i,j]*cos(W_alpha[i,j]) 
        W_cn[i] = W_cn[i] + cn3d[i,j]*sin(psi[j])*dpsi/2
        W_cs[i] = W_cs[i] + cs3d[i,j]*sin(psi[j])*dpsi/2
        W_cl[i] = W_cl[i] + cl3d[i,j]*sin(psi[j])*dpsi/2
        W_cd[i] = W_cd[i] + cd3d[i,j]*sin(psi[j])*dpsi/2
    end
end


#cn_w = 

#sum_bcoeff = 0
#    for n = 1:n_bterm
#        if rem(n,2) != 0
#            sum_bcoeff = sum_bcoeff + b_coeff[n]
#        end
#    end

#    cnc_f[i] = -2*pi*(u[i]*cos(alpha[i])/surf.uref + hdot[i]*sin(alpha[i])/surf.uref)*(sum_bcoeff)
#    sum_bdot = 0
#        for n = 1:n_bterm
#            if rem(n,2) != 0
#                sum_bdot = sum_bdot + bdot[n]
#            end
#        end
#        cnnc_f[i] = -(2*pi*surf.c/(surf.uref))*(3*sum_bdot/4)
#    end

In [ ]:
dlt = readcsv("../test/anto_cl.csv")
plot(dlt[:,1]/T,dlt[:,2])
t_th = W_mat[1][:,1]*c_d/u_d
range = round(Int,(ncyc-1)*nsteps/ncyc)+1:nsteps
tbyT = (t_th-t_th[1])/T
plot(tbyT,W_cl[range])




In [ ]:
plot(W_mat[1][:,1]*c_d/u_d,W_cl)
PyPlot.axis([0,4,0.0,0.7])

In [ ]:
plot(W_mat[1][:,1]*c_d/u_d,cd_ind/6)
#PyPlot.axis([0,120,0,1])

In [ ]:
W_mat[1]

In [ ]:
W_mat[2]

In [ ]:
### Visualise the 3D problem
fig = figure()
ax = gca(projection="3d")

plot3D(map(q->q.x, surf1.bv),map(q->q.z,surf1.bv),-3,"y",color = "black",linewidth=2.0)
plot3D(map(q->q.x, surf2.bv),map(q->q.z,surf2.bv),-2,"y",color = "black",linewidth=2.0)
plot3D(map(q->q.x, surf3.bv),map(q->q.z,surf3.bv),-1,"y",color = "black",linewidth=2.0)
plot3D(map(q->q.x, surf4.bv),map(q->q.z,surf4.bv),0, "y",color = "black",linewidth=2.0)
plot3D(map(q->q.x, surf1.bv),map(q->q.z,surf1.bv),3, "y",color = "black",linewidth=2.0)
plot3D(map(q->q.x, surf2.bv),map(q->q.z,surf2.bv),2, "y",color = "black",linewidth=2.0)
plot3D(map(q->q.x, surf3.bv),map(q->q.z,surf3.bv),1, "y",color = "black",linewidth=2.0)


scatter3D(map(q->q.x, curfield1.tev),map(q->q.z,curfield1.tev),-3,"z",s=20,c=map(q->q.s,curfield1.tev),cmap=ColorMap("jet"),edgecolors="none")
scatter3D(map(q->q.x, curfield2.tev),map(q->q.z,curfield2.tev),-2,"z",s=20,c=map(q->q.s,curfield2.tev),cmap=ColorMap("jet"),edgecolors="none")
scatter3D(map(q->q.x, curfield3.tev),map(q->q.z,curfield3.tev),-1,"z",s=20,c=map(q->q.s,curfield3.tev),cmap=ColorMap("jet"),edgecolors="none")
scatter3D(map(q->q.x, curfield4.tev),map(q->q.z,curfield4.tev),0,"z",s=20,c=map(q->q.s,curfield4.tev),cmap=ColorMap("jet"),edgecolors="none")
scatter3D(map(q->q.x, curfield1.tev),map(q->q.z,curfield1.tev),3,"z",s=20,c=map(q->q.s,curfield1.tev),cmap=ColorMap("jet"),edgecolors="none")
scatter3D(map(q->q.x, curfield2.tev),map(q->q.z,curfield2.tev),2,"z",s=20,c=map(q->q.s,curfield2.tev),cmap=ColorMap("jet"),edgecolors="none")
scatter3D(map(q->q.x, curfield3.tev),map(q->q.z,curfield3.tev),1,"z",s=20,c=map(q->q.s,curfield3.tev),cmap=ColorMap("jet"),edgecolors="none")






#axis("equal")
#ax[:view_init](20, 120)


